# Partial Reconfiguration

Starting from image v2.4, the overlay class is able to handle 
partial bitstream downloading. After a partial bitstream is
downloaded, the registers / parameters / hierarchies information
will be exposed to users, so users can easily poke the registers
associated to that partial region.

As a reminder, users can always use the following code to download
a full bitstream. Note that `BaseOverlay` is a wrapper for the 
underlying `Overlay` class.

```python
from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')
```
Then we can check the overlay information as follows:
```python
print(base.ip_dict)
print(base.interrupt_pins)
print(base.gpio_dict)
```

## 1. Use `Overlay` class for static region

The first step is to instantiate an `Overlay` object with the full bitstream.

In [1]:
from pynq import Overlay

ol = Overlay('gpio_pr.bit')

After the full bitstream has been downloaded, the partial region will be loaded
with the default Reconfigurable Module (RM). However, since the `hwh` file 
generated for the full bitstream ignores all the information in the partial 
region, no information on that partial region can be parsed. 

For the above reason, in the IP dictionary, internal IP blocks will be shown 
as memory interfaces (e.g. `gpio_0/S_AXI1`).

In [2]:
from pprint import pprint
pprint(ol.ip_dict)

{'gpio_0/S_AXI': {'addr_range': 65536,
                  'driver': <class 'pynq.overlay.DefaultIP'>,
                  'fullpath': 'gpio_0/S_AXI',
                  'gpio': {},
                  'interrupts': {},
                  'mem_id': 'S_AXI',
                  'parameters': {'C_BASEADDR': '0x41230000',
                                 'C_HIGHADDR': '0x4123FFFF',
                                 'EDK_IPTYPE': 'PERIPHERAL'},
                  'phys_addr': 1092812800,
                  'registers': {},
                  'state': None,
                  'type': 'xilinx.com:module_ref:pd_gpio_0:1.0'},
 'gpio_0/S_AXI1': {'addr_range': 65536,
                   'driver': <class 'pynq.overlay.DefaultIP'>,
                   'fullpath': 'gpio_0/S_AXI1',
                   'gpio': {},
                   'interrupts': {},
                   'mem_id': 'S_AXI1',
                   'parameters': {'C_BASEADDR': '0x41230000',
                                  'C_HIGHADDR': '0x4123FFFF',
       

                                                                     'description': 'Interrupt '
                                                                                    'Vector '
                                                                                    'Address '
                                                                                    'Register '
                                                                                    '24'}},
                                                  'size': 32},
                                     'IVAR[25]': {'access': 'read-write',
                                                  'address_offset': 356,
                                                  'description': 'Interrupt '
                                                                 'Vector '
                                                                 'Address '
                                                                 'Register 25',
       

                                                                      'bit_offset': 0,
                                                                      'bit_width': 32,
                                                                      'description': 'Interrupt '
                                                                                     'Vector '
                                                                                     'Address '
                                                                                     'Register '
                                                                                     '14'}},
                                                   'size': 32},
                                     'IVEAR[15]': {'access': 'read-write',
                                                   'address_offset': 632,
                                                   'description': 'Interrupt '
                                                       

                                                                                    'Vector '
                                                                                    'Address '
                                                                                    'Register '
                                                                                    '2'}},
                                                  'size': 32},
                                     'IVEAR[30]': {'access': 'read-write',
                                                   'address_offset': 752,
                                                   'description': 'Interrupt '
                                                                  'Vector '
                                                                  'Address '
                                                                  'Register 30',
                                                   'fields': {'IVA': {'access': 'read-write',
     

For this example, the `gpio_dict` of the overlay is empty, 
but you can check dictionaries for the interrupt, as well as
the hierarchy dictionary in the next cell.

We don't expect users to change the system interrupt controller, but
the interrupt pins may get changed when downloading a partial bitstream
onto a partial region.

Note that before we download the partial bitstream, the interrupt pins have 
full paths only to the top level of the partial region (`gpio_0` in this example).

In [3]:
pprint(ol.interrupt_controllers)
pprint(ol.interrupt_pins)

{'system_interrupts': {'index': 0, 'parent': ''}}
{'concat_interrupts/In0': {'controller': 'system_interrupts',
                           'fullpath': 'concat_interrupts/In0',
                           'index': 0},
 'concat_interrupts/In1': {'controller': 'system_interrupts',
                           'fullpath': 'concat_interrupts/In1',
                           'index': 1},
 'gpio_0/ip2intc_irpt': {'controller': 'system_interrupts',
                         'fullpath': 'gpio_0/ip2intc_irpt',
                         'index': 1},
 'gpio_0/ip2intc_irpt1': {'controller': 'system_interrupts',
                          'fullpath': 'gpio_0/ip2intc_irpt1',
                          'index': 0}}


Use the following to check the hierarchy dictionary. 
```python
pprint(ol.hierarchy_dict)
```

The hierarchy dictionary is quite a long dictionary so 
we will not show it in this notebook, but you
can try it by yourself.

## 2. Interacting with the partial region
Users have to specify the name of the partial region (hierarchical block name
where you specified your partial region). In this example, it is `gpio_0`.

Now we can reuse the `pr_download()` method with the following arguments - 
1. the partial region name,
2. the name of the partial bitstream,
3. (optional) the name of the device tree blob file if any.

In [4]:
ol.pr_download('gpio_0', 'led_5.bit')

Once the partial bitstream has been downloaded, 
you should be able to see the LEDs show the pattern `0101` (`0x5`) 
for LED 3 ~ LED 0.
Also, all the relevant dictionaries are updated.

We can also check which bitstream has been loaded.

In [5]:
pprint(ol.pr_dict)

{'gpio_0': {'dtbo': None,
            'loaded': '/home/xilinx/jupyter_notebooks/partial_reconfig/led_5.bit'}}


Notice that in the `ip_dict`, the internal IP block names have been
updated.

In [6]:
pprint(ol.ip_dict)

{'gpio_0/btns_gpio': {'addr_range': 65536,
                      'driver': <class 'pynq.lib.axigpio.AxiGPIO'>,
                      'fullpath': 'gpio_0/btns_gpio',
                      'gpio': {},
                      'interrupts': {'ip2intc_irpt': {'controller': 'system_interrupts',
                                                      'fullpath': 'gpio_0/btns_gpio/ip2intc_irpt',
                                                      'index': 0}},
                      'mem_id': 'S_AXI3',
                      'parameters': {'C_ALL_INPUTS': '1',
                                     'C_ALL_INPUTS_2': '0',
                                     'C_ALL_OUTPUTS': '0',
                                     'C_ALL_OUTPUTS_2': '0',
                                     'C_BASEADDR': '0x00000000',
                                     'C_DOUT_DEFAULT': '0x00000000',
                                     'C_DOUT_DEFAULT_2': '0x00000000',
                                     'C_FAMILY': 'zynq',
   

                       'registers': {'CIE': {'access': 'read-write',
                                             'address_offset': 20,
                                             'description': 'Clear Interrupt '
                                                            'Enables',
                                             'fields': {'INT': {'access': 'read-write',
                                                                'bit_offset': 0,
                                                                'bit_width': 2,
                                                                'description': 'Clear '
                                                                               'Interrupt '
                                                                               'Enables'}},
                                             'size': 2},
                                     'IAR': {'access': 'write-only',
                                             'address_offset': 12,


                                                  'fields': {'IVA': {'access': 'read-write',
                                                                     'bit_offset': 0,
                                                                     'bit_width': 32,
                                                                     'description': 'Interrupt '
                                                                                    'Vector '
                                                                                    'Address '
                                                                                    'Register '
                                                                                    '21'}},
                                                  'size': 32},
                                     'IVAR[22]': {'access': 'read-write',
                                                  'address_offset': 344,
                                                  '

                                     'IVEAR[12]': {'access': 'read-write',
                                                   'address_offset': 608,
                                                   'description': 'Interrupt '
                                                                  'Vector '
                                                                  'Address '
                                                                  'Register 12',
                                                   'fields': {'IVA': {'access': 'read-write',
                                                                      'bit_offset': 0,
                                                                      'bit_width': 32,
                                                                      'description': 'Interrupt '
                                                                                     'Vector '
                                                                             

                                                                  'Address '
                                                                  'Register 28',
                                                   'fields': {'IVA': {'access': 'read-write',
                                                                      'bit_offset': 0,
                                                                      'bit_width': 32,
                                                                      'description': 'Interrupt '
                                                                                     'Vector '
                                                                                     'Address '
                                                                                     'Register '
                                                                                     '28'}},
                                                   'size': 32},
                               

Pay attention to the following parameter; it indicates the default output
value from the `leds_gpio` IP block.

In [7]:
ol.ip_dict['gpio_0/leds_gpio']['parameters']['C_DOUT_DEFAULT']

'0x00000005'

You can also review the hierarchy dictionary by:

```python
pprint(ol.hierarchy_dict)
```

For interrupts, you can see that the interrupt pins are also updated. 
The full paths are now pointing to the internal GPIO blocks.

In [8]:
pprint(ol.interrupt_pins)

{'concat_interrupts/In0': {'controller': 'system_interrupts',
                           'fullpath': 'concat_interrupts/In0',
                           'index': 0},
 'concat_interrupts/In1': {'controller': 'system_interrupts',
                           'fullpath': 'concat_interrupts/In1',
                           'index': 1},
 'gpio_0/btns_gpio/ip2intc_irpt': {'controller': 'system_interrupts',
                                   'fullpath': 'gpio_0/btns_gpio/ip2intc_irpt',
                                   'index': 0},
 'gpio_0/switches_gpio/ip2intc_irpt': {'controller': 'system_interrupts',
                                       'fullpath': 'gpio_0/switches_gpio/ip2intc_irpt',
                                       'index': 1}}


You can also repeat the same process for another partial bitstream.

For example, after running the next cell, you should be able to see the onboard LEDs show
the pattern `1010` (`0xa`) for LED 3 ~ LED 0.

In [9]:
ol.pr_download('gpio_0', 'led_a.bit')

Check the currently loaded bitstream.

In [10]:
pprint(ol.pr_dict)

{'gpio_0': {'dtbo': None,
            'loaded': '/home/xilinx/jupyter_notebooks/partial_reconfig/led_a.bit'}}


Now check the new value for the default output from `leds_gpio` IP block.

In [11]:
ol.ip_dict['gpio_0/leds_gpio']['parameters']['C_DOUT_DEFAULT']

'0x0000000a'

I hope you enjoy this example.